In [1]:
import pandas as pd 
import numpy as np 
import configparser
import os

config = configparser.ConfigParser()
config.read("env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
base_path = config['DEFAULT']['base_path']
code_dir = config['DEFAULT']['code_dir']


os.chdir(code_dir)
import preprocess
os.chdir(base_path)

OHCO = ['book_id','chap_num', 'para_num', 'sent_num', 'token_num']
k = 18
all_dir_list = [f for f in os.listdir(data_home) if f.endswith('.txt')]
text_file = f"{data_home}/{all_dir_list[k-1]}"

In [2]:
book_dict = {}
all_TOKENS = pd.DataFrame()
book_data = []

all_dir_list = [f for f in os.listdir(data_home) if f.endswith('.txt')]
for book_id, book_file in enumerate(all_dir_list):
    
    book_dict[book_id+1] = preprocess.create_tokendf(f"{data_home}/{all_dir_list[book_id]}")
    book_dict[book_id+1]['token_num'].index = pd.MultiIndex.from_tuples(
                                        [(book_id+1,) + idx for idx in book_dict[book_id+1]['token_num'].index],
                                        names=['book_id'] + book_dict[book_id+1]['token_num'].index.names)
    all_TOKENS = pd.concat([all_TOKENS, book_dict[book_id+1]['token_num']])

    print(f"{str(book_id+1).zfill(2)}] {book_file} - {book_dict[book_id+1]['book_id']}")

    book_data.append((book_id+1, all_dir_list[book_id], book_dict[book_id+1]['book_id']))


LIB = pd.DataFrame(book_data, columns=['book_id','source_file_path','raw_title'])\
    .set_index('book_id').sort_index()

01] maha01.txt - Adi Parva
02] maha02.txt - Sabha Parva
03] maha03.txt - Vana Parva
04] maha04.txt - Virata Parva
05] maha05.txt - Udyoga Parva
06] maha06.txt - Bhishma Parva
07] maha07.txt - Drona Parva
08] maha08.txt - Karna Parva
09] maha09.txt - Shalya Parva
10] maha10.txt - Sauptika Parva
11] maha11.txt - Stri Parva
12] maha12.txt - Santi Parva
13] maha13.txt - Anusasana Parva
14] maha14.txt - Aswamedha Parva
15] maha15.txt - Asramavasika Parva
16] maha16.txt - Mausala Parva
17] maha17.txt - Mahaprasthanika Parva
18] maha18.txt - Svargarohanika Parva


In [3]:
all_TOKENS

pos_tuple  pos token_str term_str
book_id chap_num para_num sent_num                                           
1       1        0        0        0         (Om, NN)   NN        Om       om
                          1        0    (Having, VBG)  VBG    Having   having
                                   1     (bowed, VBN)  VBN     bowed    bowed
                                   2       (down, IN)   IN      down     down
                                   3         (to, TO)   TO        to       to
...                                               ...  ...       ...      ...
18      6        37       2        26   (Vishnu, NNP)  NNP    Vishnu   vishnu
                                   27      (like, IN)   IN      like     like
                                   28   (Vishnu, NNP)  NNP    Vishnu   vishnu
                                   29  (himself, PRP)  PRP   himself  himself
                          3        0         (``, ``)   ``        ``       ``

[2734079 rows x 4 columns]

In [8]:
book_dict.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])

0                1    \
chap_num para_num sent_num                                      
1        0        0                 (Om, NN)              NaN   
                  1            (Having, VBG)     (bowed, VBN)   
         1        0         (Ugrasrava, NNP)           (,, ,)   
                  1              (Those, DT)  (ascetics, NNS)   
                  2            (Having, VBG)      (been, VBN)   
...                                      ...              ...   
235      4        7              (Agni, NNP)       (also, RB)   
                  8            (Having, VBG)     (eaten, VBN)   
                  9                 (At, IN)       (my, PRP$)   
                  10                (', POS)      (Thus, NNS)   
                  11                 (', '')         ('', '')   

                                           2               3    \
chap_num para_num sent_num                                       
1        0        0                        NaN             NaN   
                  1                 (down, IN)        (to, TO)   
         1        0                  (the, DT)       (son, NN)   
                  1                     (,, ,)  (wishing, VBG)   
                  2         (entertained, VBN)      (with, IN)   
...                                        ...             ...   
235      4        7                     (,, ,)   (having, VBG)   
                  8                (flesh, JJ)        (in, IN)   
                  9              (command, NN)          (,, ,)   
                  10          (addressed, VBN)        (by, IN)   
                  11                       NaN             NaN   

                                        4                    5    \
chap_num para_num sent_num                                         
1        0        0                     NaN                  NaN   
                  1         (Narayana, NNP)            (and, CC)   
         1        0                (of, IN)  (Lomaharshana, NNP)   
                  1                (to, TO)           (hear, VB)   
                  2               (due, JJ)        (respect, NN)   
...                                     ...                  ...   
235      4        7             (burnt, NN)           (that, IN)   
                  8         (abundance, NN)            (and, CC)   
                  9                (ye, NN)        (heroes, NNS)   
                  10              (the, DT)    (illustrious, JJ)   
                  11                    NaN                  NaN   

                                      6                7                  8    \
chap_num para_num sent_num                                                      
1        0        0                   NaN              NaN                NaN   
                  1           (Nara, NNP)           (,, ,)          (the, DT)   
         1        0                (,, ,)  (surnamed, VBD)       (Sauti, NNP)   
                  1           (his, PRP$)  (wonderful, JJ)  (narrations, NNS)   
                  2              (by, IN)      (those, DT)         (holy, JJ)   
...                                   ...              ...                ...   
235      4        7         (forest, JJS)       (with, IN)        (its, PRP$)   
                  8           (drunk, NN)        (fat, NN)          (and, CC)   
                  9                (,, ,)         (ye, EX)        (shall, MD)   
                  10          (Agni, NNP)           (,, ,)      (Arjuna, NNP)   
                  11                  NaN              NaN                NaN   

                                       9    ...  210  211  212  213  214  215  \
chap_num para_num sent_num                  ...                                 
1        0        0                    NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN   
                  1            (most, RBS)  ...  NaN  NaN  NaN  NaN  NaN  NaN   
         1        0                 (,, ,)  ...  NaN  NaN  NaN  NaN  NaN 

In [11]:
import nltk
for package in [
                'tokenizers/punkt', 
                'taggers/averaged_perceptron_tagger', 
                'corpora/stopwords', 
                'help/tagsets'
            ]:
            
                try:
                    nltk.data.find(package)
                except IndexError:
                    nltk.download(package)

s = book_dict[5]['sent_num']
s = s.sent_str.apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))

In [12]:
s = s.stack().to_frame('pos_tuple')
s['pos'] = s.pos_tuple.apply(lambda x: x[1])
s['token_str'] = s.pos_tuple.apply(lambda x: x[0])
s['term_str'] = s.token_str.str.lower()   
s

pos_tuple  pos  token_str   term_str
chap_num para_num sent_num                                                
1        0        0        0           (OM, NN)   NN         OM         om
                  1        0       (HAVING, NN)   NN     HAVING     having
                           1       (BOWED, NNP)  NNP      BOWED      bowed
                           2         (down, RB)   RB       down       down
                           3           (to, TO)   TO         to         to
...                                         ...  ...        ...        ...
199      1        3        22       (tens, NNS)  NNS       tens       tens
                           23          (of, IN)   IN         of         of
                           24  (thousands, NNS)  NNS  thousands  thousands
                  4        0            (', '')   ''          '          '
                           1           ('', '')   ''         ''         ''

[216285 rows x 4 columns]

In [13]:
book_dict[5]['sent_num']

sent_str
chap_num para_num sent_num                                                   
1        0        0                                                        OM
                  1         HAVING BOWED down to Narayana, and Nara the mo...
         1        0         "Vaisampayana said, 'Then those valiant descen...
                  1         And those mighty monarchs of men all came to t...
                  2         And the revered and aged rulers of the earth, ...
...                                                                       ...
199      1        0         'Thus was arrayed that terrible force of the i...
                  1         And relying upon that force he battled with Su...
                  2         Besides those already named, other men by hund...
                  3         And the warriors by thousands and ten thousand...
                  4                                                        '"

[10109 rows x 1 columns]